In [ ]:
%pip install sklearn
%pip install pandas
%pip install numpy
%pip install requests
%pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import datetime 
import json
from wacc import Wacc
from dcf import getFreeCashFlow, dcf_calc,linear_regression_prediction, linear_regression_model, extract_fcf

# Present value formula

$$PV=FV\frac{1}{(1+r)^n}$$


* **PV** =	present value
* **FV** =	future value
* **r**	 =	rate of return
* **n**	 =	number of periods

The formula answer to this questions:
1. **What do I need to invest today to have a specific sum of money at a future date?**
2. **Is this investment-worthy enough?**

In [ ]:
# Present value example
#Variable
capital_future= 10000
years = 5
capital_now = 7000
interest_rate1 = 0.06
interest_rate2 = 0.15

print(f'THE PROBLEM: Someone wants propose you to invest {capital_now} to get {capital_future} in {years} years. How to take the decision?!')

print("\n-*-*-\n")

PV1 = capital_5yr / pow(1+interest_rate1, years)
PV2 = capital_5yr / pow(1+interest_rate2, years)

print(f'You need {PV1} to gain {capital_future} in {years} years with a {interest_rate1*100}% interest rate' )
if capital_now < PV1:
    print(f' > {capital_now} is less than {PV1}. With {interest_rate1*100}% interest rate is MORE convenient accept {capital_future} in {years} years')
else:
    print(f' > {capital_now} is more than {PV1}. With {interest_rate1*100}% interest rate is NOT convenient accept {capital_future} in {years} years')
print("\n-*-*-\n")
print(f'You need {PV2} to gain {capital_future} in {years} years with a {interest_rate2*100}% interest rate' )
if capital_now < PV2:
    print(f' > {capital_now} is less than {PV2}. With {interest_rate2*100}% interest rate is MORE convenient accept {capital_future} in {years} years')
else:
    print(f' > {capital_now} is more than {PV2}. With {interest_rate2*100}% interest rate is NOT convenient accept {capital_future} in {years} years')

# Financialmodelingprep

Register here to get the APIKEY [financialmodelingprep](https://site.financialmodelingprep.com/register)
Insert the APIKEY into the settings file

# Variable initialiazation

In [ ]:
company = 'BABA'
skey = ''

with open("settings.json", 'r') as j:
    skey=json.load(j)["fml_secretkey"]

fcf = getFreeCashFlow(company, skey)
stock= yf.Ticker(company)
stock_info = stock.info
previous_close = stock_info["regularMarketPreviousClose"]
sharesOutstanding = stock_info["sharesOutstanding"]

# The weighted average cost of capital (WACC)

The weighted average cost of capital is a common way to determine required rate of return because it expresses, in a single number, the return that both bondholders and shareholders demand in order to provide the company with capital. A firm’s WACC is likely to be higher if its stock is relatively volatile or if its debt is seen as risky because investors will demand greater returns. 

The weighted average cost of capital is a common way to determine required rate of return because it expresses, in a single number, the return that both bondholders and shareholders demand in order to provide the company with capital. A firm’s WACC is likely to be higher if its stock is relatively volatile or if its debt is seen as risky because investors will demand greater returns. [Investopedia](https://www.investopedia.com/terms/w/wacc.asp)

$$WACC = (kd*(1-ETR)*\frac{debt}{(equity+debt)}) + (ke*\frac{equity}{equity+debt})$$

* **kd** = cost of debt
* **ke** = cost of equity
* **etr** = effective tax rate
* **debt** = debt market value
* **equity** = equity market value

[What is WACC - Youtube](https://www.youtube.com/watch?v=0inqw9cCJnM)


In [ ]:
wacc = Wacc(skey) 
wacc_company = wacc.calc_wacc(company)

print('wacc of ' + company + ' is ' + str((wacc_company*100))+'%')

# Discounted Cash Flow (DFC)

Discounted cash flow (DCF) is a valuation method used to estimate the value of an investment based on its expected future cash flows. DCF analysis attempts to figure out the value of an investment today, based on projections of how much money it will generate in the future. This applies to the decisions of investors in companies or securities, such as acquiring a company or buying a stock, and for business owners and managers looking to make capital budgeting or operating expenditures decisions.

$$DCF = \sum_n\frac{CF}{(1+wacc)^n} $$

* **CF** = cash flow during the period n
* **wacc** = Weighted Average Cost of Capital
* **n** = period n

In [ ]:
model = linear_regression_model(fcf)
fcf_pred = linear_regression_prediction(model, 0, 5)
dcf = dcf_calc(fcf_pred, wacc_company, len(fcf_pred), multiplier=1)

fair_price = dcf / sharesOutstanding

print(f'The fair price calculated is {fair_price}. Current price is {previous_close}')

# Usefull concept 

### Cost of debt
he cost of debt is the effective interest rate that a company pays on its debts, such as bonds and loans. The cost of debt can refer to the before-tax cost of debt, which is the company’s cost of debt before taking taxes into account, or the after-tax cost of debt. The key difference in the cost of debt before and after taxes lies in the fact that interest expenses are tax-deductible.

### Cost of equity
The cost of equity is the return that a company requires to decide if an investment meets capital return requirements. Firms often use it as a capital budgeting threshold for the required rate of return. A firm’s cost of equity represents the compensation that the market demands in exchange for owning the asset and bearing the risk of ownership. The traditional formula for the cost of equity is the dividend capitalization model and the capital asset pricing model (CAPM).

### Risk free
The risk-free rate of return is the theoretical rate of return of an investment with zero risk. The risk-free rate represents the interest an investor would expect from an absolutely risk-free investment over a specified period of time.
The so-called "real" risk-free rate can be calculated by subtracting the current inflation rate from the yield of the Treasury bond matching your investment duration.

thanks to [Investopedia](https://www.investopedia.com)